In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [2]:
from models.encoders import LinearEncoder
from models.decoders import LinearDecoder

class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
    
        image_size = (1, 28, 28)

        # encoder part
        self.encoder = LinearEncoder(image_size= image_size, hidden_dims = [h_dim1, h_dim2], latent_dim = z_dim)
        # decoder part
        self.decoder = LinearDecoder(image_size= image_size, hidden_dims = [h_dim1, h_dim2], latent_dim = z_dim)
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
    
    def forward(self, x):
        mu, log_var = self.encoder(x)
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [3]:
vae

VAE(
  (encoder): LinearEncoder(
    (encoder): Sequential(
      (0): Flatten(start_dim=1, end_dim=-1)
      (1): Linear(in_features=784, out_features=512, bias=True)
      (2): ReLU()
      (3): Linear(in_features=512, out_features=256, bias=True)
      (4): ReLU()
    )
    (z_mean): Linear(in_features=256, out_features=2, bias=True)
    (z_log_var): Linear(in_features=256, out_features=2, bias=True)
  )
  (decoder): LinearDecoder(
    (decoder): Sequential(
      (0): Linear(in_features=2, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=784, bias=True)
      (5): Sigmoid()
    )
  )
)

In [4]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [5]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [6]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [7]:
for epoch in range(1, 51):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 545.061641
Train Epoch: 1 [10000/60000 (17%)]	Loss: 205.232266
Train Epoch: 1 [20000/60000 (33%)]	Loss: 187.883086
Train Epoch: 1 [30000/60000 (50%)]	Loss: 170.861211
Train Epoch: 1 [40000/60000 (67%)]	Loss: 168.322148
Train Epoch: 1 [50000/60000 (83%)]	Loss: 166.432129
====> Epoch: 1 Average loss: 183.5698
====> Test set loss: 165.7393
Train Epoch: 2 [0/60000 (0%)]	Loss: 166.262461
Train Epoch: 2 [10000/60000 (17%)]	Loss: 168.122090
Train Epoch: 2 [20000/60000 (33%)]	Loss: 159.623359
Train Epoch: 2 [30000/60000 (50%)]	Loss: 153.408662
Train Epoch: 2 [40000/60000 (67%)]	Loss: 163.386973
Train Epoch: 2 [50000/60000 (83%)]	Loss: 159.701924
====> Epoch: 2 Average loss: 161.0267
====> Test set loss: 157.5183
Train Epoch: 3 [0/60000 (0%)]	Loss: 150.580127
Train Epoch: 3 [10000/60000 (17%)]	Loss: 151.639678
Train Epoch: 3 [20000/60000 (33%)]	Loss: 156.784238
Train Epoch: 3 [30000/60000 (50%)]	Loss: 156.874307
Train Epoch: 3 [40000/60000 (67%)]	Loss: 153.74

In [ ]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample_' + '.png')

In [ ]:
with torch.no_grad():
    images = next(iter(test_loader))[0].cuda()
    recon, _, _ = vae(images)
    save_image(images.view(100, 1, 28, 28), './samples/real_' + '.png')
    save_image(recon.view(100, 1, 28, 28), './samples/recon_' + '.png')